In [1]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from matplotlib import pyplot
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Dataset/test.csv')

In [3]:
df1 = pd.read_csv('Dataset/train.csv')

In [4]:
targets = df1[['pet_id','breed_category','pet_category']]

In [5]:
df1 = df1.drop('breed_category', axis = True)

In [6]:
df1= df1.drop('pet_category', axis = True)

In [7]:
test = pd.concat([df1,df])

In [8]:
test['condition'].fillna(1.0, inplace=True)

In [9]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    return(res)

In [10]:
test = encode_and_bind(test, 'color_type')

In [11]:
test.drop('color_type',axis = 1, inplace = True)

In [12]:
test['height(m)']=test['height(cm)'].apply(lambda x:x*(.01))

In [13]:
train_df=test[:18834]

In [14]:
test_df= test[18834:]

In [15]:
train_df = pd.concat([train_df,targets], axis =1)

In [16]:
train_df.drop('height(cm)', axis = 1, inplace = True)
test_df.drop('height(cm)', axis = 1, inplace = True)

C:\Users\Ashish Bisht\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [17]:
train_df.drop('issue_date', axis = 1, inplace = True)
train_df.drop('listing_date', axis = 1, inplace=True)
test_df.drop('issue_date', axis = 1, inplace = True)
test_df.drop('listing_date', axis = 1, inplace=True)

In [18]:
train_df=train_df.drop(df.columns[0], axis=1)

In [19]:
init_train_df = train_df.drop('pet_category', axis = 1)

In [20]:
init_train_df

,condition,length(m),X1,X2,color_type_Agouti,color_type_Apricot,color_type_Black,color_type_Black Brindle,color_type_Black Smoke,color_type_Black Tabby,...,color_type_Tan,color_type_Torbie,color_type_Tortie,color_type_Tortie Point,color_type_Tricolor,color_type_White,color_type_Yellow,color_type_Yellow Brindle,height(m),breed_category
0,2.0,0.80,13,9,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0778,0.0
1,1.0,0.72,13,9,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0.1419,0.0
2,1.0,0.15,15,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.4090,2.0
3,1.0,0.62,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0.1782,0.0
4,2.0,0.50,18,4,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0.1106,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18829,2.0,0.44,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0.2736,0.0
18830,1.0,0.73,15,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.1425,2.0
18831,0.0,0.99,13,9,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.2813,1.0
18832,0.0,0.55,13,9,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0.4482,1.0


In [21]:
X1_train = init_train_df.drop('breed_category', axis = 1)
Y1_test = init_train_df['breed_category']

In [22]:
params = {
          
          
          "num_leaves" : [10,20,30,40,50,60,70,80,90,100,150,200],
          "max_depth": [1,2,3, 4, 5],
          
          }

In [23]:
lgb = LGBMClassifier( n_estimators=600, learning_rate = 0.01, bagging_fraction = 0.9,  # subsample
          feature_fraction = 0.9,  # colsample_bytree
          bagging_freq = 5,        # subsample_freq
          bagging_seed = 2018,
          verbosity = 1,
                     objective = 'multi:softmax',
                    num_class=3, nthread=1)

In [24]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedKFold
folds = 5
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(lgb, param_distributions=params, n_iter=param_comb, n_jobs=4, cv=skf.split(X1_train,Y1_test), verbose=1, random_state=1001 )


In [25]:
random_search.fit(X1_train,Y1_test)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=4)]: Done  25 out of  25 | elapsed:   26.9s finished


RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x0000019B318F16C8>,
                   estimator=LGBMClassifier(bagging_fraction=0.9,
                                            bagging_freq=5, bagging_seed=2018,
                                            feature_fraction=0.9,
                                            learning_rate=0.01,
                                            n_estimators=600, nthread=1,
                                            num_class=3,
                                            objective='multi:softmax',
                                            verbosity=1),
                   n_iter=5, n_jobs=4,
                   param_distributions={'max_depth': [1, 2, 3, 4, 5],
                                        'num_leaves': [10, 20, 30, 40, 50, 60,
                                                       70, 80, 90, 100, 150,
                                                       200]},
                   random_state=1001, verbose=1)

In [26]:
test1 = test_df.drop('pet_id', axis =1)

In [27]:
y_test = random_search.predict(test1)

In [28]:
sec_train_df = train_df.drop('breed_category', axis = 1)

In [29]:
X2_train = sec_train_df.drop('pet_category', axis = 1)
Y2_test = sec_train_df['pet_category']

In [30]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5],
        
        }

In [31]:
lgm = LGBMClassifier(learning_rate=0.02, n_estimators=600, objective = 'multi:softmax',
                    num_class=4, nthread=1)

In [32]:
folds = 5
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search_1 = RandomizedSearchCV(lgm, param_distributions=params, n_iter=param_comb, n_jobs=4, cv=skf.split(X2_train,Y2_test), verbose=3, random_state=1001 )


In [33]:
random_search_1.fit(X2_train,Y2_test)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  25 out of  25 | elapsed:   25.9s finished


RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x0000019B3188E248>,
                   estimator=LGBMClassifier(learning_rate=0.02,
                                            n_estimators=600, nthread=1,
                                            num_class=4,
                                            objective='multi:softmax'),
                   n_iter=5, n_jobs=4,
                   param_distributions={'colsample_bytree': [0.6, 0.8, 1.0],
                                        'gamma': [0.5, 1, 1.5, 2, 5],
                                        'max_depth': [3, 4, 5],
                                        'min_child_weight': [1, 5, 10],
                                        'subsample': [0.6, 0.8, 1.0]},
                   random_state=1001, verbose=3)

In [34]:
y_test1 = random_search_1.predict(test1)

In [35]:
results_df = pd.DataFrame(data={'pet_id':test_df['pet_id'], 'breed_category':y_test,'pet_category':y_test1})
results_df.to_csv('submission_LGBM.csv', index=False)

In [36]:
y_test1

array([2, 1, 2, ..., 2, 4, 2], dtype=int64)